In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
!pip install pyodbc

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 280 kB 4.4 MB/s 
  Created wheel for pyodbc: filename=pyodbc-4.0.32-cp37-cp37m-linux_x86_64.whl size=287363 sha256=1a40975fa2e067ea669150c72c2832b9336f53f3c9ec911e9df6d62ffe3a7571
  Stored in directory: /root/.cache/pip/wheels/2e/9c/da/8652fd42e0f662015554f00a9e96fe4f438dfd1ef59787879e
Successfully built pyodbc


In [3]:
!pip install eurostat

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 45 kB 1.5 MB/s 
     |████████████████████████████████| 49 kB 5.3 MB/s 
  Created wheel for jsonpath-rw: filename=jsonpath_rw-1.4.0-py3-none-any.whl size=15147 sha256=9f3e73cde42d01c594fbd3feef7535e69c3d78b48e5be25f9d82cdca4d854d37
  Stored in directory: /root/.cache/pip/wheels/58/88/2a/8d619cf38d7cf939e54b6ccdece05d31b64b3eb419c11d1ed3
Successfully built jsonpath-rw


In [4]:
!apt-get install virtuoso-opensource

Reading package lists... Done
Building dependency tree       
Reading state information... Done
The following package was automatically installed and is no longer required:
  libnvidia-common-460
Use 'apt autoremove' to remove it.
The following additional packages will be installed:
  libvirtodbc0 virtuoso-opensource-6.1 virtuoso-opensource-6.1-bin
  virtuoso-opensource-6.1-common virtuoso-server virtuoso-vad-conductor
  virtuoso-vsp-startpage
Suggested packages:
  virtuoso-vad-doc virtuoso-vad-demo virtuoso-vad-tutorial
  virtuoso-vad-rdfmappers virtuoso-vad-sparqldemo virtuoso-vad-syncml
  virtuoso-vad-bpel virtuoso-vad-isparql virtuoso-vad-ods virtuoso-vad-dbpedia
  virtuoso-vad-facetedbrowser
The following NEW packages will be installed:
  libvirtodbc0 virtuoso-opensource virtuoso-opensource-6.1
  virtuoso-opensource-6.1-bin virtuoso-opensource-6.1-common virtuoso-server
  virtuoso-vad-conductor virtuoso-vsp-startpage
0 upgraded, 8 newly installed, 0 to remove and 42 not upgraded.


In [5]:
import gensim
import pandas as pd
import numpy as np
import re
import pyodbc
import dateutil.parser as parser

c = pyodbc.connect('DRIVER=/usr/lib/odbc/virtodbc.so;HOST=lod.csd.auth.gr:1111;UID=kimon;PWD=RkhvQYZ442e2JVXLHdtW;DATABASE=ESTAT')
#set encoding
c.setdecoding(pyodbc.SQL_CHAR, encoding='latin-1')
c.setencoding(encoding="latin-1")
cursor = c.cursor()


def load_table(cursor,query):
  cursor.execute(query)
  t1 = cursor.fetchall()
  df = pd.DataFrame.from_records(t1, columns=[x[0] for x in cursor.description])
  return df

In [6]:
#OECD

query =      """SELECT id,article_id,term,url,definition,context,theme,related,related_url,last_update,source_publ FROM ESTAT.V1.OECD_Glossary"""

OECDGlossary = load_table(cursor,query)

query =      """SELECT id,ESTAT_theme,ESTAT_sub_theme,OECD_themes FROM ESTAT.V1.Eurostat_OECD_themes"""

OECDThemes = load_table(cursor,query)

In [7]:
#Map OECD
listHelper = []
for name, row in OECDGlossary.iterrows():
  helper = ""
  row[2] = row[2].replace("?", "___")
  nameTerm = str(re.sub("[^a-zA-Z\d%]", ' ', row[2]).replace("\'", "").replace("\"", "").replace("\\", "").replace("-", "_").strip().replace(" ", "_")).lower()
  nameTerm = nameTerm.replace("____", "").replace("___", "").replace("__", "")
  listHelper.append((str(row[1]), nameTerm))

f = open("OECD.ttl", "a")
for name, row in OECDGlossary.iterrows():
  helper = ""
  row[2] = row[2].replace("?", "____")
  nameTerm = str(re.sub("[^a-zA-Z\d%]", ' ', row[2]).replace("\'", "").replace("\"", "").replace("\\", "").strip().replace(" ", "_")).lower()
  nameTerm = nameTerm.replace("____", "").replace("___", "").replace("__", "")
  if nameTerm != "":
    f.write("oecd:" + nameTerm + " rdf:type estat:OECDTerm.\n")
    f.write("oecd:" + nameTerm + " estat:term \"" + str(re.sub("[^a-zA-Z\d%]", ' ', row[2]).replace("\'", "").replace("\"", "").replace("\\", "").strip()) +"\"^^xsd:string.\n")
    f.write("oecd:" + nameTerm + " estat:id \"" + str(row[1]) +"\"^^xsd:integer.\n")
    if str(row[3]) != "":
      f.write("oecd:" + nameTerm + " estat:hasURL \"" + str(row[3]) + "\"^^xsd:anyURI .\n")
    if str(row[4]) != "":
      f.write("oecd:" + nameTerm + " estat:definition \"" + str(re.sub("[^a-zA-Z\d%]", ' ', row[4]).replace("\'", "").replace("\"", "").replace("\\", "").strip()) +"\"^^xsd:string.\n")
    if str(re.sub("[^a-zA-Z\d%]", ' ', row[5]).replace("\'", "").replace("\"", "").replace("\\", "").strip()) != "":
      f.write("oecd:" + nameTerm + " estat:context \"" + str(re.sub("[^a-zA-Z\d%]", ' ', row[5]).replace("\'", "").replace("\"", "").replace("\\", "").strip()) +"\"^^xsd:string.\n")

    if str(row[6]) != "":
      themeOECD = str(re.sub("[^a-zA-Z\d%]", '_', row[6]).replace("\'", "").replace("\"", "").replace("\\", "").strip().replace(" ", "_")).lower().replace("____", "").replace("___", "").replace("__", "")
      if themeOECD[-1] == "_":
        themeOECD = themeOECD[:-1]
      f.write("oecd:" + nameTerm + " estat:hasOECDTheme oecd:" + themeOECD +".\n")
      f.write("oecd:" + themeOECD + " rdf:type estat:OECDTheme.\n")
      f.write("oecd:" + themeOECD + " estat:title \"" + str(re.sub("[^a-zA-Z\d%]", ' ', row[6]).replace("\'", "").replace("\"", "").replace("\\", "").strip()).lower() + "\"^^xsd:string.\n")

    if str(row[8]) != "":
      helperSplit = str(row[8]).split(",")
      for uri in helperSplit:
        if "?ID=" in str(uri):
          uri = uri.split("?ID=")[1]
          for l in listHelper:
            if str(uri) == str(l[0]):
              f.write("oecd:" + nameTerm + " estat:relatedTerm oecd:" + l[1] + " .\n")

    if str(row[9]) != "":
      date = parser.parse(str(row[9]))
      f.write("oecd:" + nameTerm + " estat:dateUpdated \"" + date.isoformat() + "\"^^xsd:dateTime.\n")
    if str(row[10]) != "":
      f.write("oecd:" + nameTerm + " estat:sourcePublication \"" + str(re.sub("[^a-zA-Z\d%]", ' ', row[10]).replace("\'", "").replace("\"", "").replace("\\", "").strip()).lower() +"\"^^xsd:string.\n")

  
f.close()


In [8]:
f = open("OECD.ttl", "a")
for name, row in OECDThemes.iterrows():
  eurostatTheme = str(re.sub("[^a-zA-Z\d%]", ' ', row[1]).replace("\'", "").replace("\"", "").replace("\\", "").replace("-", "_").strip().replace(" ", "_")).lower().replace("____", "").replace("___", "").replace("__", "")
  flag = 0
  f.write("estatdata:" + eurostatTheme + " rdf:type estat:EurostatTheme.\n")
  f.write("estatdata:" + eurostatTheme + " estat:title \"" + str(re.sub("[^a-zA-Z\d%]", ' ', row[1]).replace("\'", "").replace("\"", "").replace("\\", "").strip()).lower() + "\"^^xsd:string.\n\n") 
  for theme in row[3].split(";"):
    oecdTheme = str(re.sub("[^a-zA-Z\d%]", ' ', theme).replace("\'", "").replace("\"", "").replace("\\", "").replace("-", "_").strip().replace(" ", "_")).lower().replace("____", "").replace("___", "").replace("__", "")
    if oecdTheme != "":
      f.write("estatdata:" + eurostatTheme + " estat:relatedTheme oecd:" + oecdTheme + ".\n")
      flag = 1
  if flag == 1:
    if str(row[2]) not in row[3].split(";"):
      subtheme = str(re.sub("[^a-zA-Z\d%]", ' ', row[2]).replace("\'", "").replace("\"", "").replace("\\", "_").replace("-", "_").strip().replace(" ", "_")).lower().replace("____", "").replace("___", "").replace("__", "")
      f.write("estatdata:" + subtheme + " rdf:type estat:EurostatTheme.\n")
      f.write("estatdata:" + subtheme + " estat:title \"" + str(re.sub("[^a-zA-Z\d%]", ' ', row[2]).replace("\'", "").replace("\"", "").replace("\\", "").strip()) +"\"^^xsd:string.\n")
      f.write("estatdata:" + eurostatTheme + " estat:hasSubTheme estatdata:" + subtheme + ".\n")
      for theme in row[3].split(";"):
        oecdTheme = str(re.sub("[^a-zA-Z\d%]", ' ', theme).replace("\'", "").replace("\"", "").replace("\\", "").replace("-", "_").strip().replace(" ", "_")).lower().replace("____", "").replace("___", "").replace("__", "")
        f.write("estatdata:" + subtheme + " estat:relatedTheme oecd:" + oecdTheme + ".\n")

f.close()